# 1. Captura de tweets

## 1.1 Constantes

In [ ]:
import pandas as pd

# Rutas y nombre de archivos de los tweets capturados
tweets_local_path = 'data/twitter_raw/'
tweets_hdfs_path = '/user/testing86/twitter/'
tweets_filename = 'tweets.json'

tweets_local_path_filename = tweets_local_path + tweets_filename
tweets_hdfs_path_filename = tweets_hdfs_path + tweets_filename

# Ruta Tweets capturados
twitter_results_path = 'data/twitter_results/'

# Tecnologías más populares del último año
stack_overflow_results_path = "data/stack_overflow_results/"
popular_tags_2017_filename = 'list_popular_tags_2017.csv'
popular_tags_2017 = pd.read_csv(stack_overflow_results_path + popular_tags_2017_filename).tag.tolist()


# Palabras no válidas
not_valid_country = 'null'
not_valid_language = ['null', 'und']
not_valid_coordenates = 999.9

# Otras constantes
numberOfResults = 10
others_local_path = 'data/others/'

## 1.2 Autenticación del usuario de Twitter para realizar la captura

In [ ]:
import tweepy

consumer_key = 'sM12C9ouJEADz1w2F7hfKerAU'
consumer_secret = 'pTAdQF9oTtoy63saHTnUmbrv9v2FYi2LYYyH0UqXZOPA9Vpxqk'

access_token = '83638117-1lYKb3OTUiYIPibnm9IZ1vadthCRquCelOCCqZoax'
access_token_secret = 'WVtdNK6lKyDJKhFqQCroHYi4zHSZANB0bRdkKsWK9NuCk'

auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.secure = True
auth.set_access_token(access_token, access_token_secret)

api = tweepy.API(auth)
print ("Autenticación correcta")

## 1.3 Captura y filtrado de tweets

### 1.3.1 Funcíon de captura y filtrado

In [ ]:
from tweepy import Stream
from tweepy.streaming import StreamListener
from tweepy.api import API
import json

class TwitterListener(StreamListener):    
    def __init__(self, path, keywords, capture_type = 'byTime', capture_time_minutes = 5, capture_number = 100):
        self.api = api or API()
        self.path = path
        self.keywords = keywords
        self.capture_type = capture_type
        self.start_time = time.time()
        self.stop_time = 0
        self.counter = 0
        self.capture_time_minutes = capture_time_minutes * 60
        self.tweets_to_capture = capture_number
        self.capture_finished = False
        self.error = False
        
        print('Captura de tweets iniciada.')
        print('Palabras clave = ', end = '')
        for i in range(len(self.keywords)):
            if i == (len(self.keywords)-1):
                print(self.keywords[i], end ='\n')
            else:
                print(self.keywords[i], end =', ')
            
        if self.capture_type == 'byTime':
            print('Tiempo de captura: {} min\n'.format(self.capture_time_minutes//60))
            print('\rTiempo restante: {} min {:02d} s.     Tweets capturados: {}'
                  .format(self.capture_time_minutes//60, self.capture_time_minutes%60, self.counter), end = '', flush = True)
        elif self.capture_type == 'byTweets':
            print('Tweets a capturar: {}\n'.format(self.tweets_to_capture))
            print('\rTweets capturados: {} de {}.     Tiempo transcurrido: {} min {:02d} s'
                  .format(self.counter, self.tweets_to_capture, 0, 0), end = '', flush = True)
        else:
            print('Error: {} no es un tipo de captura válido.'.format(self.capture_type))
            self.capture_finished = True
            self.error = True
              
    def on_status(self, status):
        
        toSave={}
        if self.capture_type == 'byTime':
            if (time.time() - self.start_time) < self.capture_time_minutes:
                remaining_time_min = (self.capture_time_minutes//60) - ((int(time.time() -self.start_time))//60) - 1
                remaining_time_s = 59 - (int(time.time() -self.start_time))%60
                print('\rTiempo restante: {} min {:02d} s.     Tweets capturados: {}'
                      .format(remaining_time_min, remaining_time_s, self.counter), end = '', flush = True)
            else:
                self.stop_time = time.time()
                self.capture_finished = True
        elif self.capture_type == 'byTweets':
            if self.counter < self.tweets_to_capture:
                elapsed_time_min = int(time.time() -self.start_time)//60
                elapsed_time_s = int(time.time() -self.start_time)%60
                print('\rTweets capturados: {} de {}.     Tiempo transcurrido: {} min {:02d} s'
                      .format(self.counter, self.tweets_to_capture, elapsed_time_min, elapsed_time_s), end = '', flush = True)
            else:
                self.stop_time = time.time()
                self.capture_finished = True
                
        if not self.capture_finished:
            if status.text:
                texto = [word for word in status.text.lower().split() if word in self.keywords]
                if texto:
                    self.counter += 1

                    country = 'null'
                    coordinates = {
                            'Longitude':999.9,
                            'Latitude':999.9}
                    lang = 'null'
                    created_at = 'null'

                    if status.place:
                        country = status.place.country
                    if status.lang:
                        lang = status.lang
                    if status.created_at:
                        created_at = status.created_at
                    #if status.geo:
                    #if status.user.geo_enabled and status.geo:
                    #    coordinates = status.geo
                    if status.coordinates is not None:
                        coordinates = {
                            'Longitude':status.coordinates['coordinates'][0],
                            'Latitude':status.coordinates['coordinates'][1]}

                    toSave = {
                        'Tag':texto[0],
                        'Language':lang,
                        'Country':country,
                        'Coordinates':coordinates,
                        'Created_at':created_at}
                    try:
                        #with open('/home/testing86/Proyecto/data/tweets_pp.json', 'a') as f:
                        with open(self.path, 'a') as f:
                            stringToWrite = json.dumps(toSave,default=str)+'\n'
                            f.write(stringToWrite)

                            #json.dump(toSave,f)
                            return True
                    except BaseException as e:
                        print("Error on_data: %s" % str(e))
                    return True
        else:
            if not self.error:
                print('\rCaptura de tweets finalizada.')
                if self.capture_type == 'byTime':
                    print('Se han capturado {} tweets en {} min'.format(self.counter,(self.capture_time_minutes//60)))
                elif self.capture_type == 'byTweets':
                    print('Se han capturado {} tweets en {} min y {} s.'
                          .format(self.counter,(int(time.time() -self.start_time))//60, (int(time.time() -self.start_time))%60))

                if (self.counter > 0):
                    print('Un tweet cada {0:.3f} s'.format((self.stop_time - self.start_time)/self.counter))
            return False
 
    def on_error(self, status):
        print(status)
        return True

### 1.3.2 Realizar la captura de tweets en base a las tecnologías más populares del último año

#### 1.3.2.1 Configurar la captura de tweets

In [ ]:
# Habilitar o deshabilitar la captura de tweets
capture_tweets = False
# Tipo de captura ('byTweets' o 'byTime')
capture_type = 'byTweets'
# Parámetros de la captura
keywords = popular_tags_2017
tweets_to_capture = 100
minutes_to_capture = 1    

#### 1.3.2.2 Iniciar la captura de tweets

In [ ]:
if capture_tweets:
    # Límite por número de tweets
    if capture_type == 'byTweets':
        twitterStream= Stream(auth, TwitterListener(tweets_local_path_filename, keywords, 
                                                    capture_type = capture_type, 
                                                    capture_number = tweets_to_capture),async=True)
        twitterStream.filter(track=keywords)
    # Límite por tiempo
    elif capture_type == 'byTime':
        twitterStream= Stream(auth, TwitterListener(tweets_local_path_filename, keywords, 
                                                    capture_type = capture_type, 
                                                    capture_time_minutes = minutes_to_capture),async=True)
        twitterStream.filter(track=keywords)

## 1.4 Ingesta a HDFS

In [ ]:
if capture_tweets:
    ! hdfs dfs -put -f {tweets_local_path_filename} {tweets_hdfs_path_filename}

# 2. Lectura de los tweets capturados

## 2.1 Lectura del archivo alojado en HDFS

In [ ]:
tweets_df = spark.read.json(tweets_hdfs_path_filename).cache()

## 2.2 Número de tweets capturados

In [ ]:
# Número de tweets
tweets_count = tweets_df.count()

### 2.2.1 Guardar a un fichero

In [ ]:
tweets_count_filename = 'tweets_count.csv'
with open(twitter_results_path + tweets_count_filename, 'w') as f:
    f.write(str(tweets_count))

# 3. Procesamientos de datos

## 3.1 Popularidad actual de las tecnologías más populares del año anterior

In [ ]:
# Agrupar, contar y hacer la media de los tweets
from pyspark.sql.functions import desc, col, count, format_number

ranking_popular_tags_df = (tweets_df.groupBy('Tag')
               .agg(count('Tag').alias('Menciones')).sort(desc('Menciones'))
               .withColumn('Promedio', format_number(col('Menciones')/tweets_count * 100,2).cast('Float')).cache())

### 3.1.1 Guardar los resultados a un fichero

In [ ]:
ranking_popular_tags_filename = 'ranking_popular_tags.csv'
ranking_popular_tags_df.toPandas().to_csv(twitter_results_path 
                                           + ranking_popular_tags_filename, header=True, index=False)

### 3.1.2 Variables auxiliares empleadas en la obtención de resultados

In [ ]:
# Lista de las tecnologías más populares actualmente
current_ranking_tag_list = ranking_popular_tags_df.select('Tag').rdd.flatMap(lambda x: x).collect()

# Lista del número de menciones de cada tecnología
current_ranking_tag_count_list = ranking_popular_tags_df.select('Menciones').rdd.flatMap(lambda x: x).collect()

## 3.2 Tecnologías por países

### 3.2.1 Tweets en los que se especifica el país

In [ ]:
countries_df = (tweets_df.filter(tweets_df.Country.isin(not_valid_country) == False)).cache()
number_tweets_with_country = countries_df.count()

#### 3.2.1.1 Guardar los resultados a un fichero

In [ ]:
number_tweets_with_country_filename = 'number_tweets_with_country.csv'
with open(twitter_results_path + number_tweets_with_country_filename, 'w') as f:
    f.write(str(number_tweets_with_country))

### 3.2.2 Ranking de los países en los que las tecnologías son más mencionadas

In [ ]:
ranking_countries_df = (countries_df
                        .groupBy('Country').agg(count('Country').alias('Menciones'))
                        .sort(desc('Menciones'))
                        .withColumn('Promedio', 
                                    format_number(col('Menciones')/number_tweets_with_country * 100,2)
                                    .cast('Float')).cache())

#### 3.2.2.1 Guardar los resultados a un fichero

In [ ]:
ranking_countries_filename = 'ranking_countries.csv'
ranking_countries_df.limit(10).toPandas().to_csv(twitter_results_path 
                                           + ranking_countries_filename, header=True, index=False)

## 3.3 Tecnologías por idiomas

### 3.3.1 Tweets en los que se especifica el idioma

In [ ]:
languages_df = (tweets_df.filter(tweets_df.Language.isin(not_valid_language) == False)).cache()
number_tweets_with_language = languages_df.count()

### 3.3.2 Ranking de los idiomas más empleados

In [ ]:
ranking_languages_df = (languages_df
                       .groupBy('Language').agg(count('Language').alias('Menciones'))
                       .sort(desc('Menciones'))
                       .withColumn('Promedio', format_number(col('Menciones')/number_tweets_with_language * 100,2)
                                   .cast('Float')).cache())

#### 3.3.2.1 Carga del diccionario de idiomas

In [ ]:
import json
abbreviate_languages_filename = 'idiomas.json'
with open(others_local_path + abbreviate_languages_filename) as f:
    abbreviate_languages = json.load(f)

#### 3.3.2.2 Conversión de la abreviatura del idioma al nombre del idioma

In [ ]:
temporal_pd_pf = ranking_languages_df.limit(10).toPandas()
for i in range(len(temporal_pd_pf)):
    temporal_pd_pf.loc[i,'Language'] = abbreviate_languages[temporal_pd_pf.Language[i]]

#### 3.3.2.3 Guardar los resultados a un fichero

In [ ]:
ranking_languages_filename = 'ranking_languages.csv'
temporal_pd_pf.to_csv(twitter_results_path + ranking_languages_filename, header=True, index=False)

#### 3.3.2.4 Diccionario de idiomas y número de tweets

In [ ]:
# Lista de idiomas
languages_list = ranking_languages_df.select('Language').rdd.flatMap(lambda x: x).collect()

# Lista del número de tags por idioma
languages_tags_counts_list = ranking_languages_df.select('Menciones').rdd.flatMap(lambda x: x).collect()

# Diccionario que relaciona el idioma con su número de tags
dict_language_tagsCounts = {}
for i in range(len(languages_list)):
    dict_language_tagsCounts[languages_list[i]] = languages_tags_counts_list[i]

### 3.3.3 Idiomas más empleados por tecnología

In [ ]:
from pyspark.sql.functions import udf,lit
from pyspark.sql.types import *

def average_tag_per_language(language, tag_count):
    counts_language = dict_language_tagsCounts[language]
    return (tag_count/counts_language)*100

udf_average_tag_per_language = udf(average_tag_per_language, DoubleType())

ranking_language_by_tag = []
for i in range(len(current_ranking_tag_list)):
    ranking_language_by_tag.append(languages_df.filter(languages_df.Tag == current_ranking_tag_list[i])
                                   .groupBy('Language').agg(count('Language').alias('Menciones')).sort(desc('Menciones'))
                                   .withColumn('PromedioTag',
                                               format_number(col('Menciones')/current_ranking_tag_count_list[i] * 100,2)
                                               .cast('Float'))
                                   .withColumn('PromedioIdioma',
                                               format_number(udf_average_tag_per_language
                                                             ('Language','Menciones'),2).cast('Float'))
                                   .withColumn('Tecnologia',lit(current_ranking_tag_list[i])).cache())

#### 3.3.3.1 Guardar los resultados a ficheros convirtiendo la abreviatura del idiomas en su nombre

In [ ]:
rank = 1

for i in range(len(current_ranking_tag_list)):
    popular_languages_by_tag_rank_filename = 'popular_languages_by_tag_rank_' + str(rank) + '.csv'
    temporal_pd_pf = ranking_language_by_tag[i].limit(10).toPandas()
    # Traducción de la abreviatura
    for i in range(len(temporal_pd_pf)):
        temporal_pd_pf.loc[i,'Language'] = abbreviate_languages[temporal_pd_pf.Language[i]]
    # Guardar los resultados
    temporal_pd_pf.to_csv(twitter_results_path + popular_languages_by_tag_rank_filename, header=True, index=False)
                                      
    rank +=1

## 3.4  Tweets geolocalizados por coodenadas

In [ ]:
from pyspark.sql.functions import hour

remove_not_valid = 999.9
geo_tweets_df = (tweets_df.filter(tweets_df.Coordinates.Latitude != remove_not_valid)
                 .select(tweets_df.Coordinates.Latitude.alias('Latitude'), 
                         tweets_df.Coordinates.Longitude.alias('Longitude'),
                         hour(tweets_df.Created_at).alias('Hour'),
                         tweets_df.Tag).cache())

### 3.4.1 Guardar los resultados a un fichero

In [ ]:
geo_tweets_filename = 'geo_tweets.csv'
geo_tweets_df.toPandas().to_csv(twitter_results_path + geo_tweets_filename, header=True, index=False)